<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/cda-2021/blob/main/notebooks/cda_2021_06_08_lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We need to instal the `sampleSelection` package to use original Heckman's sample selection model.

In [1]:
install.packages("sampleSelection")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘matrixStats’, ‘RcppArmadillo’, ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘conquer’, ‘sp’, ‘openxlsx’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘zoo’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘maptools’, ‘rio’, ‘lme4’, ‘sandwich’, ‘car’, ‘lmtest’, ‘maxLik’, ‘miscTools’, ‘systemfit’, ‘Formula’, ‘VGAM’, ‘mvtnorm’




In [2]:
library(sampleSelection)

Loading required package: maxLik

Loading required package: miscTools


Please cite the 'maxLik' package as:
Henningsen, Arne and Toomet, Ott (2011). maxLik: A package for maximum likelihood estimation in R. Computational Statistics 26(3), 443-458. DOI 10.1007/s00180-010-0217-1.

If you have questions, suggestions, or comments regarding the 'maxLik' package, please use a forum or 'tracker' at maxLik's R-Forge site:
https://r-forge.r-project.org/projects/maxlik/



In order to use Heckman's sample selection model we should use function `heckit` from `sampleSelection` package. 

The function have the following arguments:

+ selection -- selection equation 
+ outcome -- outcome equation 
+ data -- where we specify dataset
+ method -- how the model should be estimated (`twostep` or `ml`)


```{r}
m2 <- heckit(selection ~ married + children + education + age,
             wage ~  education + age, 
             data = women,
             method = "ml")
summary(m2)
```

In [5]:
data(Mroz87)
Mroz87$kids  <- ( Mroz87$kids5 + Mroz87$kids618 > 0 )
head(Mroz87, n = 3)
tail(Mroz87, n = 3)

,lfp,hours,kids5,kids618,age,educ,wage,repwage,hushrs,husage,⋯,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,wifecoll,huscoll,kids
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,⋯,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<fct>,<fct>,<lgl>
1,1,1610,1,0,32,12,3.3540,2.65,2708,34,⋯,0.7215,12,7,5,0,14,10.91006,FALSE,FALSE,TRUE
2,1,1656,0,2,30,12,1.3889,2.65,2310,30,⋯,0.6615,7,7,11,1,5,19.49998,FALSE,FALSE,TRUE
3,1,1980,1,3,35,12,4.5455,4.04,3072,40,⋯,0.6915,12,7,5,0,15,12.03991,FALSE,FALSE,TRUE


,lfp,hours,kids5,kids618,age,educ,wage,repwage,hushrs,husage,⋯,mtr,motheduc,fatheduc,unem,city,exper,nwifeinc,wifecoll,huscoll,kids
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,⋯,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<fct>,<fct>,<lgl>
751,0,0,0,0,43,12,0,0,2383,43,⋯,0.7515,10,3,7.5,0,4,9.952,FALSE,FALSE,FALSE
752,0,0,0,0,60,12,0,0,1705,55,⋯,0.6215,12,12,14.0,1,15,24.984,FALSE,FALSE,FALSE
753,0,0,0,3,39,9,0,0,3120,48,⋯,0.6915,7,7,11.0,1,12,28.363,FALSE,FALSE,TRUE


Let's start with a simple linear model where we focus only on women who participate in the labour market. 

In [6]:
model1 <- lm(formula = wage ~ exper + I(exper^2) + educ + city,
             data = Mroz87,
             subset = lfp == 1)
summary(model1)


Call:
lm(formula = wage ~ exper + I(exper^2) + educ + city, data = Mroz87, 
    subset = lfp == 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.6021 -1.6012 -0.4787  0.8950 21.2762 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.5609920  0.9288390  -2.757  0.00608 ** 
exper        0.0324982  0.0615864   0.528  0.59800    
I(exper^2)  -0.0002602  0.0018378  -0.142  0.88747    
educ         0.4809623  0.0668679   7.193 2.91e-12 ***
city         0.4492741  0.3177735   1.414  0.15815    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 3.111 on 423 degrees of freedom
Multiple R-squared:  0.1248,	Adjusted R-squared:  0.1165 
F-statistic: 15.08 on 4 and 423 DF,  p-value: 1.569e-11


Now, let's create Heckman's model using `heckit` function

In [7]:
model2 <- heckit(selection = lfp ~ age + I(age^2) + faminc + kids + educ,
                 outcome = wage ~ exper + I(exper^2) + educ + city, 
                 data = Mroz87,
                 method = "ml")
summary(model2)

--------------------------------------------
Tobit 2 model (sample selection model)
Maximum Likelihood estimation
Newton-Raphson maximisation, 5 iterations
Return code 8: successive function values within relative tolerance limit (reltol)
Log-Likelihood: -1581.258 
753 observations (325 censored and 428 observed)
13 free parameters (df = 740)
Probit selection equation:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4.120e+00  1.401e+00  -2.942 0.003368 ** 
age          1.840e-01  6.587e-02   2.794 0.005345 ** 
I(age^2)    -2.409e-03  7.723e-04  -3.119 0.001886 ** 
faminc       5.680e-06  4.416e-06   1.286 0.198782    
kidsTRUE    -4.506e-01  1.302e-01  -3.461 0.000568 ***
educ         9.528e-02  2.315e-02   4.115  4.3e-05 ***
Outcome equation:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.9630242  1.1982209  -1.638    0.102    
exper        0.0278683  0.0615514   0.453    0.651    
I(exper^2)  -0.0001039  0.0018388  -0.056    0.955    
educ 

Standard model

```
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -2.5609920  0.9288390  -2.757  0.00608 ** 
exper        0.0324982  0.0615864   0.528  0.59800    
I(exper^2)  -0.0002602  0.0018378  -0.142  0.88747    
educ         0.4809623  0.0668679   7.193 2.91e-12 ***
city         0.4492741  0.3177735   1.414  0.15815    
```

Heckman's model
```
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.9630242  1.1982209  -1.638    0.102    
exper        0.0278683  0.0615514   0.453    0.651    
I(exper^2)  -0.0001039  0.0018388  -0.056    0.955    
educ         0.4570051  0.0732299   6.241 7.33e-10 ***
city         0.4465290  0.3159209   1.413    0.158   
```